In [ ]:
import globus_sdk

from globus_sdk.scopes import TransferScopes, GCSCollectionScopeBuilder


CLIENT_ID = "4dc65ee6-20f2-4e5a-a7c4-76ddd27fb2d5"
auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)

# Replace these with your own collection UUIDs
PERLMUTTER = "9d6d994a-6d04-11e5-ba46-22000b92c6ec"
MyMac =  "753f24e6-b6d6-11ed-a504-1f2a3a60e896"
source_collection_id = PERLMUTTER
dest_collection_id = PERLMUTTER

scopes = TransferScopes.make_mutable("all")
src_access_scope = GCSCollectionScopeBuilder(source_collection_id).make_mutable("data_access")
dst_access_scope = GCSCollectionScopeBuilder(dest_collection_id).make_mutable("data_access")
scopes.add_dependency(src_access_scope)
scopes.add_dependency(dst_access_scope)


# requested_scopes specifies a list of scopes to request
# instead of the defaults, only request access to the Transfer API
auth_client.oauth2_start_flow(requested_scopes=scopes)
authorize_url = auth_client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")

auth_code = input("Please enter the code here: ").strip()
tokens = auth_client.oauth2_exchange_code_for_tokens(auth_code)
transfer_tokens = tokens.by_resource_server["transfer.api.globus.org"]

# construct an AccessTokenAuthorizer and use it to construct the
# TransferClient
transfer_client = globus_sdk.TransferClient(
    authorizer=globus_sdk.AccessTokenAuthorizer(transfer_tokens["access_token"])
)



Please go to this URL and login:

https://auth.globus.org/v2/oauth2/authorize?client_id=4dc65ee6-20f2-4e5a-a7c4-76ddd27fb2d5&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall%5Bhttps%3A%2F%2Fauth.globus.org%2Fscopes%2F9d6d994a-6d04-11e5-ba46-22000b92c6ec%2Fdata_access+https%3A%2F%2Fauth.globus.org%2Fscopes%2F9d6d994a-6d04-11e5-ba46-22000b92c6ec%2Fdata_access%5D&state=_default&response_type=code&code_challenge=QWPM6YPlqh9xj5hqL3HJpIFHVvx2H3SmROprA0DADjo&code_challenge_method=S256&access_type=online



In [2]:
import globus_sdk
from globus_sdk.scopes import TransferScopes, GCSCollectionScopeBuilder
GCSCollectionScopeBuilder?

Init signature:
GCSCollectionScopeBuilder(
    resource_server: 'str',
    *,
    known_scopes: 'ScopeBuilderScopes' = None,
    known_url_scopes: 'ScopeBuilderScopes' = None,
) -> 'None'
Docstring:     
A ScopeBuilder with a named property for the GCS data_access scope.
"data_access" is a scope on GCS Collections. The resource_server string should
be the GCS Collection ID.

**Examples**

>>> sb = GCSCollectionScopeBuilder("xyz")
>>> da_scope = sb.data_access
>>> https_scope = sb.https
File:           ~/miniconda3/envs/prefect/lib/python3.12/site-packages/globus_sdk/scopes/data.py
Type:           type
Subclasses:     

In [ ]:


# create a Transfer task consisting of one or more items
task_data = globus_sdk.TransferData(
    source_endpoint=source_collection_id, destination_endpoint=dest_collection_id,
    fail_on_quota_errors=True,
    preserve_timestamp=True,
    notify_on_succeeded=True,
    notify_on_failed=True,
    notify_on_inactive = True
)
# task_data.add_item(
#     "/global/homes/r/rjnathan/mytest.txt",  # source
#     "/~/minimal-example-transfer-script-destination.txt", # dest
#     recursive=False
# )

#)
task_data.add_item(
    "/global/cfs/cdirs/m3239/ip2data/bella_pw_data/PW/Y2024/03-Mar/24_0328/analysis/scanfiles.tar",  # source
    "/global/homes/r/rjnathan/mytest/scanfile.tar", # dest
    recursive=False
)

# submit, getting back the task ID
task_doc = transfer_client.submit_transfer(task_data)
task_id = task_doc["task_id"]
print(f"submitted transfer, task_id={task_id}")

In [ ]:

transfer_client = globus_sdk.TransferClient(
    authorizer=globus_sdk.AccessTokenAuthorizer(transfer_tokens["access_token"])
)
task_list = transfer_client.task_list(limit=20)
for task in task_list:
    print(task['task_id'],task['status'])